In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
application_df = application_df.drop(columns=['EIN', 'NAME'])

application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
app_counts = application_df.APPLICATION_TYPE.value_counts()

app_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [4]:
replace_application = list(app_counts[app_counts < 200].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [5]:
classification_counts = application_df.CLASSIFICATION.value_counts()
classification_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [6]:
replace_class = list(classification_counts[classification_counts < 2000].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
Other     6062
C1200     4837
Name: CLASSIFICATION, dtype: int64

In [7]:
# Generate our categorical variable lists
#  YOUR CODE GOES HERE
application_cat = application_df.dtypes[application_df.dtypes == 'object'].index.tolist()
print(application_cat)

['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS']


In [8]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
#  YOUR CODE GOES HERE
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

C:\Users\Graham\anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [9]:
# Merge one-hot encoded features and drop the originals
#  YOUR CODE GOES HERE
application_df = application_df.merge(encode_df, left_index=True, right_index=True)
application_df = application_df.drop(application_cat, axis=1)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [10]:
# Split our preprocessed data into our features and target arrays
#  YOUR CODE GOES HERE
X = application_df.drop('IS_SUCCESSFUL',axis=1) 
y = application_df.IS_SUCCESSFUL
# Split the preprocessed data into a training and testing dataset
#  YOUR CODE GOES HERE
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, stratify=y)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [18]:
## First Attempt - Changing number of hidden nodes

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# *** ATTEMPT 1 *** 
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 24
hidden_nodes_layer2 = 12

nn = tf.keras.models.Sequential()

# First hidden layer
#  YOUR CODE GOES HERE
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features,activation='relu'))

# Second hidden layer
#  YOUR CODE GOES HERE
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))


# Output layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 24)                1008      
                                                                 
 dense_1 (Dense)             (None, 12)                300       
                                                                 
 dense_2 (Dense)             (None, 1)                 13        
                                                                 
Total params: 1,321
Trainable params: 1,321
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Compile the model
#  YOUR CODE GOES HERE
nn.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [14]:
# Train the model
#  YOUR CODE GOES HERE
fit_model = nn.fit(X_train, y_train, epochs=150)

Epoch 1/150
804/804 [==============================] - 1s 1ms/step - loss: 29964.4863 - accuracy: 0.5064
Epoch 2/150
804/804 [==============================] - 1s 1ms/step - loss: 34307.6953 - accuracy: 0.5044
Epoch 3/150
804/804 [==============================] - 1s 966us/step - loss: 16796.0098 - accuracy: 0.5092
Epoch 4/150
804/804 [==============================] - 1s 962us/step - loss: 24927.4219 - accuracy: 0.5175
Epoch 5/150
804/804 [==============================] - 1s 952us/step - loss: 13133.4746 - accuracy: 0.5161
Epoch 6/150
804/804 [==============================] - 1s 939us/step - loss: 8231.3701 - accuracy: 0.5002
Epoch 7/150
804/804 [==============================] - 1s 991us/step - loss: 24401.0820 - accuracy: 0.5013
Epoch 8/150
804/804 [==============================] - 1s 1ms/step - loss: 21397.0996 - accuracy: 0.5079
Epoch 9/150
804/804 [==============================] - 1s 1ms/step - loss: 11024.8936 - accuracy: 0.5126
Epoch 10/150
804/804 [========================

In [19]:
# Evaluate Attempt 1
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f'Loss: {model_loss}, Accuracy: {model_accuracy}')

268/268 - 0s - loss: 1.0007 - accuracy: 0.7155 - 323ms/epoch - 1ms/step
Loss: 1.0007424354553223, Accuracy: 0.7154518961906433


In [ ]:
## Attempt 2 - Trying an additional hidden layer

In [20]:

number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1=30
hidden_nodes_layer2=20
hidden_nodes_layer3=10

nn = tf.keras.models.Sequential()

In [21]:
# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features,activation='relu'))

# Second layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2,activation='relu'))

# Third layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3,activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

# Check model structure
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 30)                1260      
                                                                 
 dense_4 (Dense)             (None, 20)                620       
                                                                 
 dense_5 (Dense)             (None, 10)                210       
                                                                 
 dense_6 (Dense)             (None, 1)                 11        
                                                                 
Total params: 2,101
Trainable params: 2,101
Non-trainable params: 0
_________________________________________________________________


In [23]:
# compile model
nn.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

# Train model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 1ms/step - loss: 0.5860 - accuracy: 0.7083
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5596 - accuracy: 0.7261
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5569 - accuracy: 0.7275
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5550 - accuracy: 0.7271
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5530 - accuracy: 0.7295
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5526 - accuracy: 0.7299
Epoch 7/100
804/804 [==============================] - 1s 989us/step - loss: 0.5517 - accuracy: 0.7304
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5513 - accuracy: 0.7311
Epoch 9/100
804/804 [==============================] - 1s 996us/step - loss: 0.5508 - accuracy: 0.7285
Epoch 10/100
804/804 [==============================] - 1s 981us/step - loss: 0.5495 - 

804/804 [==============================] - 1s 1ms/step - loss: 0.5413 - accuracy: 0.7365
Epoch 82/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5415 - accuracy: 0.7354
Epoch 83/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5413 - accuracy: 0.7359
Epoch 84/100
804/804 [==============================] - 1s 992us/step - loss: 0.5412 - accuracy: 0.7365
Epoch 85/100
804/804 [==============================] - 1s 988us/step - loss: 0.5410 - accuracy: 0.7371
Epoch 86/100
804/804 [==============================] - 1s 983us/step - loss: 0.5411 - accuracy: 0.7368
Epoch 87/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5413 - accuracy: 0.7357
Epoch 88/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5411 - accuracy: 0.7365
Epoch 89/100
804/804 [==============================] - 1s 998us/step - loss: 0.5410 - accuracy: 0.7352
Epoch 90/100
804/804 [==============================] - 1s 990us/step - loss: 0.5415 - 

In [24]:
# Evaluate attempt 2
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f'Loss: {model_loss}, Accuracy: {model_accuracy}')

268/268 - 0s - loss: 0.5601 - accuracy: 0.7257 - 413ms/epoch - 2ms/step
Loss: 0.5600776672363281, Accuracy: 0.7257142663002014


In [31]:
## Attempt 3 - Try a Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf_model = RandomForestClassifier(n_estimators=1000, random_state=42)

rf_model = rf_model.fit(X_train_scaled, y_train)

y_pred = rf_model.predict(X_test_scaled)
print(f'Random forest accuracy: {accuracy_score(y_test,y_pred):.3f}')

Random forest accuracy: 0.710


In [32]:
# Export to HDF5
nn.save('AlphabetSoupCharity_Optimization.h5')